In [1]:
# baseball-predictor.ipynb based on hello-world-predictor
# Objective: Read the dataset, do a simple prediction for games on a given date using data prior to that date, then
#            and record some stats on how well it did.
# 
#
# 12/21/19, Alexis: Currently this "helloworld" predictor is very simple. It does the following:
#  1. Reads in the 2010 season as the baseline games dataset. It does some simple selection
#     of columns and generates a few calculated fields.
#  2. It prompts the user to specify 'game day' which are the games it is to predict.
#  3. It calculates the avg net number of runs for the home team (when playing at home) and
#     the avg net number of runs for the visiting team (when playing away).
#  4. For a given game, the predicted winner is the team with the greatest avg net runs at
#     home or away (as applicable).
#  5. The prediction is compared with the actual results that day, with % correct reported.
#
#  Proposed next steps:
#  1. Integrate Venkat's "concat" capability to combine datasets across year blocks to result
#     in a much larger dataset. Note that for development we may want to keep the dataset
#     smaller size so that it doesn't take a long time to run.
#  2. Move this out of jupyter notebook into standard Python and embed the prediction logic
#     into a function that can be called repeatedly with different dates.
#  3. Run trials with large numbers of dates to product large numbers of predictions and
#     results.
#  4. Write results to a file. Possibly generate some plot of results as a function of
#     training set window size.
#  5. Start experimenting with diffent prediction functions, initially across individual
#     factors, and then with multiple factors.
#  6. Consider a statistically meaningful regresssion analysis to select factors and training
#     set window size, by factor.
#  7. If someone has energy, consider using a web API to hit a website with current day
#     game schedule so we can predict games more recent than the dataset.
#
#
# 12/21/19, Venkat : Modified the code and included the below changes
#  1. Included the logic to combine all data files into one data frame
#  2. Included cleanup logic to clean missing data rows if any such rows exists
#
# 1/4/20, Alexis: Extend from "hello-world" to a useful predictor harness.
#  1. Implement a window to constrain the lookback period to n gamedays.
#
#
# 1/5/20, Venkat: Building the new prediction algorithm.
#                 Calculate Avg Net runs only for head-head teams, and see if it improves the forecst accuracy 




In [2]:
# Modules
import os
import csv
import pprint
import pandas as pd
import glob
import pprint
import numpy as np

files = glob.glob("../datasets/Final_Data_Files/GL*.csv")
files.sort()

def reader(f):
    df = pd.read_csv(f, index_col=False, header=None)  
    df.columns = [("Col_"+str(i)) for i in range(1,df.shape[1]+1)]       

    return df

#season_df = pd.concat([reader(f) for f in files], keys=files)
season_df = pd.concat([reader(f) for f in files])

# season_df

In [3]:
season_df.shape # Useful to make sure we don't loose rows when adding column headers.

(19437, 161)

In [4]:
season_df = season_df.rename(columns={'Col_1':'Date',
                                     'Col_4':'Visiting Team',
                                     'Col_5':'Visiting League',
                                     'Col_7':'Home Team',
                                     'Col_8':'Home League',
                                     'Col_10':'Visiting Score',
                                     'Col_11':'Home Score'})
season_df = season_df[['Date', 'Visiting Team', 'Visiting League', 'Home Team', 'Home League',
                       'Visiting Score','Home Score']]
# season_df

In [5]:
# Identify incomplete rows
season_df.count()

Date               19437
Visiting Team      19437
Visiting League    19437
Home Team          19437
Home League        19437
Visiting Score     19437
Home Score         19437
dtype: int64

In [6]:
# Create a dataframe of unique game days with the game ID for the first game

# season_df.head()
gamedays = pd.Series(season_df['Date'].unique())
print(gamedays.head())
print(gamedays.tail())
print(type(gamedays.iloc[0]))

def gamedays_offset(base_date, n): # returns new game date offset by n
    base_date_index = gamedays[gamedays==base_date].index[0]
    if ((n + base_date_index) < 0) or n + base_date_index >= len(gamedays):
        raise ValueError(f"Attempting to caluclate a game date outside the range of the dataset.")
        return(0)  # Out of range
    else:
        new_index = base_date_index + n
#         print(f"new_index={new_index}")
        return gamedays.iloc[new_index]

# This is a manual test of the gamedays_offset() function.
print(gamedays_offset(20120406, -100))
    

0    20100404
1    20100405
2    20100406
3    20100407
4    20100408
dtype: int64
1433    20170927
1434    20170928
1435    20170929
1436    20170930
1437    20171001
dtype: int64
<class 'numpy.int64'>
20110622


In [7]:
# Drop all rows with missing information
season_df = season_df.dropna(how='any')
season_df.count()

Date               19437
Visiting Team      19437
Visiting League    19437
Home Team          19437
Home League        19437
Visiting Score     19437
Home Score         19437
dtype: int64

In [8]:
# Create a column 
season_df['Home Winner'] = season_df['Home Score'] > season_df['Visiting Score']
season_df['Winner'] = np.where(season_df['Home Score'] > season_df['Visiting Score'], season_df['Home Team'], season_df['Visiting Team'])
season_df['V NetRuns'] = season_df['Visiting Score'] - season_df['Home Score']
season_df['H NetRuns'] = - season_df['V NetRuns']
season_df['V Wins'] = ""
season_df['H Wins'] = ""
season_df['Predict Home Wins?'] = ""
season_df['Prediction Correct?'] = ""

season_df.head()

,Date,Visiting Team,Visiting League,Home Team,Home League,Visiting Score,Home Score,Home Winner,Winner,V NetRuns,H NetRuns,V Wins,H Wins,Predict Home Wins?,Prediction Correct?
0,20100404,NYA,AL,BOS,AL,7,9,True,BOS,-2,2,,,,
1,20100405,MIN,AL,ANA,AL,3,6,True,ANA,-3,3,,,,
2,20100405,CLE,AL,CHA,AL,0,6,True,CHA,-6,6,,,,
3,20100405,DET,AL,KCA,AL,8,4,False,DET,4,-4,,,,
4,20100405,SEA,AL,OAK,AL,5,3,False,SEA,2,-2,,,,


In [9]:
# Ask the user how big of a window of prior game days should we evaluate.
good_value = False
while not good_value:
    lookback_n = int(input(f"How many gamedays back do you want the predictor to look? "))
    good_value = lookback_n > 0 and lookback_n < len(gamedays)

How many gamedays back do you want the predictor to look? 100


In [10]:
dataset_start_date = int(season_df['Date'].min())
start_date = gamedays_offset(dataset_start_date, lookback_n)
end_date = season_df['Date'].max()
print(f"dataset_start_date: {dataset_start_date}")
print(f"start_date: {start_date}")
print(f"end_date: {end_date}")

good_date = False
while not good_date:
    game_day = int(input(f"What is the game day (YYYYMMDD) you want us to predict (between {start_date} and {end_date}: "))
#    game_day = 20100715
    if game_day >= start_date and game_day <= end_date:
        game_day_df = season_df.loc[(season_df['Date'] == (game_day))]
        if len(game_day_df)==0:
            print("Sorry, no games are scheduled for that day")
        else:
            lookback_start_day = gamedays_offset(game_day, -lookback_n)
            lookback_end_day = gamedays_offset(game_day, -1)
            print(f"Lookback window starts {lookback_start_day} - {lookback_end_day}")
            train_df = season_df.loc[((season_df['Date'] < (game_day)) & (season_df['Date'] >= (lookback_start_day)))]
            gds = str(game_day)
            print(f"Nice! There are {len(game_day_df)} games on {gds[0:4]}-{gds[4:6]}-{gds[6:]}.")
            print(f"  We also have {len(train_df)} games in our training set, which should be plenty!")
            good_date = True
    else:
        print("Sorry, the date you selected is outside the range of our dataset.")

train_df.head()

dataset_start_date: 20100404
start_date: 20100716
end_date: 20171001
What is the game day (YYYYMMDD) you want us to predict (between 20100716 and 20171001: 20151004
Lookback window starts 20150622 - 20151003
Nice! There are 16 games on 2015-10-04.
  We also have 1367 games in our training set, which should be plenty!


,Date,Visiting Team,Visiting League,Home Team,Home League,Visiting Score,Home Score,Home Winner,Winner,V NetRuns,H NetRuns,V Wins,H Wins,Predict Home Wins?,Prediction Correct?
1046,20150622,HOU,AL,ANA,AL,3,4,True,ANA,-1,1,,,,
1047,20150622,DET,AL,CLE,AL,8,5,False,DET,3,-3,,,,
1048,20150622,CHA,AL,MIN,AL,2,13,True,MIN,-11,11,,,,
1049,20150622,PHI,NL,NYA,AL,11,8,False,PHI,3,-3,,,,
1050,20150622,KCA,AL,SEA,AL,4,1,False,KCA,3,-3,,,,


In [11]:
# We now have the set of games we want to predict, and the training set defined.
# game_day_df2 = game_day_df.copy(deep=True)
game_day_df2 = game_day_df.copy(deep=True)
game_day_df2.head()

,Date,Visiting Team,Visiting League,Home Team,Home League,Visiting Score,Home Score,Home Winner,Winner,V NetRuns,H NetRuns,V Wins,H Wins,Predict Home Wins?,Prediction Correct?
2413,20151004,NYA,AL,BAL,AL,4,9,True,BAL,-5,5,,,,
2414,20151004,DET,AL,CHA,AL,6,0,False,DET,6,-6,,,,
2415,20151004,BOS,AL,CLE,AL,1,3,True,CLE,-2,2,,,,
2416,20151004,KCA,AL,MIN,AL,6,1,False,KCA,5,-5,,,,
2417,20151004,OAK,AL,SEA,AL,2,3,True,SEA,-1,1,,,,


In [12]:
v_teams = game_day_df2['Visiting Team'].to_list()
v_wins =0
h_wins =0
for team in v_teams:
    v_wins = (len(train_df[(train_df['Visiting Team'] == team) & (train_df['Winner'] == team) ])) - (len(train_df[(train_df['Visiting Team'] == team) & (train_df['Winner'] != team) ]))
    
    game_day_df2.loc[(game_day_df2['Visiting Team'] == team), ['V Wins']] = v_wins
    v_wins =0

h_teams = game_day_df2['Home Team'].to_list()
  
for team in h_teams:
    h_wins = (len(train_df[(train_df['Home Team'] == team) & (train_df['Winner'] == team)])) - (len(train_df[(train_df['Home Team'] == team) & (train_df['Winner'] != team)]))
    #game_day_df2['V Wins'] = np.where(game_day_df2['Visiting Team'] == team1, v_wins, 0)
    game_day_df2.loc[(game_day_df2['Home Team'] == team), ['H Wins']] = h_wins
    h_wins =0
    
game_day_df2['Predict Home Wins?'] = game_day_df2['H Wins'] > game_day_df2['V Wins']

game_day_df2

,Date,Visiting Team,Visiting League,Home Team,Home League,Visiting Score,Home Score,Home Winner,Winner,V NetRuns,H NetRuns,V Wins,H Wins,Predict Home Wins?,Prediction Correct?
2413,20151004,NYA,AL,BAL,AL,4,9,True,BAL,-5,5,5,6,True,
2414,20151004,DET,AL,CHA,AL,6,0,False,DET,6,-6,-10,-3,True,
2415,20151004,BOS,AL,CLE,AL,1,3,True,CLE,-2,2,-3,4,True,
2416,20151004,KCA,AL,MIN,AL,6,1,False,KCA,5,-5,3,3,False,
2417,20151004,OAK,AL,SEA,AL,2,3,True,SEA,-1,1,-8,-5,True,
2418,20151004,TOR,AL,TBA,AL,3,12,True,TBA,-9,9,6,-1,False,
2419,20151004,ANA,AL,TEX,AL,2,9,True,TEX,-7,7,-4,5,True,
2420,20151004,HOU,AL,ARI,NL,3,5,True,ARI,-2,2,-16,-4,True,
2421,20151004,SLN,NL,ATL,NL,0,6,True,ATL,-6,6,9,-3,False,
2422,20151004,SLN,NL,ATL,NL,0,2,True,ATL,-2,2,9,-3,False,


In [13]:
# Evaluate the predictions
game_day_df2['Prediction Correct?'] = game_day_df2['Predict Home Wins?'] == game_day_df2['Home Winner']
game_day_df2_correct = game_day_df2.loc[(game_day_df2['Prediction Correct?']),:]
print(f"{game_day}: {len(game_day_df2)} games with {len(game_day_df2_correct)} predicted correctly.")
print(f"{round((len(game_day_df2_correct)/len(game_day_df2)*100.),1)}%")
game_day_df2

20151004: 16 games with 11 predicted correctly.
68.8%


,Date,Visiting Team,Visiting League,Home Team,Home League,Visiting Score,Home Score,Home Winner,Winner,V NetRuns,H NetRuns,V Wins,H Wins,Predict Home Wins?,Prediction Correct?
2413,20151004,NYA,AL,BAL,AL,4,9,True,BAL,-5,5,5,6,True,True
2414,20151004,DET,AL,CHA,AL,6,0,False,DET,6,-6,-10,-3,True,False
2415,20151004,BOS,AL,CLE,AL,1,3,True,CLE,-2,2,-3,4,True,True
2416,20151004,KCA,AL,MIN,AL,6,1,False,KCA,5,-5,3,3,False,True
2417,20151004,OAK,AL,SEA,AL,2,3,True,SEA,-1,1,-8,-5,True,True
2418,20151004,TOR,AL,TBA,AL,3,12,True,TBA,-9,9,6,-1,False,False
2419,20151004,ANA,AL,TEX,AL,2,9,True,TEX,-7,7,-4,5,True,True
2420,20151004,HOU,AL,ARI,NL,3,5,True,ARI,-2,2,-16,-4,True,True
2421,20151004,SLN,NL,ATL,NL,0,6,True,ATL,-6,6,9,-3,False,False
2422,20151004,SLN,NL,ATL,NL,0,2,True,ATL,-2,2,9,-3,False,False
